In [ ]:
import json
import pandas as pd
from github import Github
import os
import datetime

In [ ]:
g = Github(os.environ["GITHUB_API_TOKEN"])

In [ ]:
with open("20230921-skbc.json") as f:
    data = json.load(f)

In [ ]:
data[0]

In [ ]:
df = pd.DataFrame.from_dict(
    {
        "path": [d["path"] for d in data],
        "repo": [d["repository"]["nameWithOwner"] for d in data],
    }
)
df = df.set_index(["repo", "path"])

In [ ]:
uniq_repos = {
    v
    for v in df.index.get_level_values("repo")
    if not v.lower().startswith(("apc524-", "scikit-build"))
}

In [ ]:
stars = {repo: g.get_repo(repo).stargazers_count for repo in uniq_repos}

In [ ]:
sorted_stars = {k: v for k, v in sorted(stars.items(), key=lambda x: -x[1])}

In [ ]:
sorted_stars

In [ ]:
df["stars"] = df.index.get_level_values("repo").map(lambda x: sorted_stars.get(x, None))

In [ ]:
df = df.dropna()

In [ ]:
df.loc[:, "stars"] = df["stars"].astype(int)

In [ ]:
df = df.sort_values("stars", ascending=False)

In [ ]:
df[["stars"]].to_csv("20230921-skbc-wstars.csv")